In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

tf.__version__

'2.12.0'

In [ ]:
import pandas as pd
import numpy as np


def makediff(df):
    res = df.copy()
    res['diff'] = df.groupby('id')['Timestamp'].diff()
    res['diff'].fillna(res.groupby('id')['diff'].transform('mean'), inplace=True)
    res['diff'] *= 80
    res['diff'] = np.log2(res['diff']) #조정 변수
    return res


def makeCSV(path, flag):
    df = pd.read_csv(path)
    df = df.dropna()
    df.rename(columns = {'Arbitration_ID' : 'id'}, inplace = True)
    df = makediff(df)
    temp = df['Data'].str.split(expand=True)
    temp.columns = ['d0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7']
    temp = temp.fillna('-1')
    if flag:
        df['Class'] = 'Normal'
        df['SubClass'] = 'Normal'
    df = pd.concat([df.loc[:, 'id'], temp, df.loc[:,['diff', 'Class', 'SubClass']]], axis=1)
    df.to_csv(path[:-4] + '_new.csv', index=False)
    return


# makeCSV('/content/drive/MyDrive/ADD/challenge/0_Preliminary/0_Training/Pre_train_D_0.csv', 1)

df = pd.read_csv('/content/drive/MyDrive/ADD/challenge/0_Preliminary/0_Training/Pre_train_D_0_new.csv')
df.loc[: ,'id':'d7'] = df.loc[: ,'id':'d7'].apply(lambda x: x.astype(str).map(lambda x: int(x, 16)))
df.loc[: ,'id':'d7'] /= 256
df = df[['id', 'diff', 'Class', 'SubClass']]
free = np.array(df)


# makeCSV('/content/drive/MyDrive/ADD/challenge/0_Preliminary/0_Training/Pre_train_D_1.csv', 0)

df = pd.read_csv('/content/drive/MyDrive/ADD/challenge/0_Preliminary/0_Training/Pre_train_D_1_new.csv')
df.loc[: ,'id':'d7'] = df.loc[: ,'id':'d7'].apply(lambda x: x.astype(str).map(lambda x: int(x, 16)))
df.loc[: ,'id':'d7'] /= 256
df = df[['id', 'diff', 'Class', 'SubClass']]
test1 = np.array(df)


# makeCSV('/content/drive/MyDrive/ADD/challenge/0_Preliminary/0_Training/Pre_train_D_2.csv', 0)

df = pd.read_csv('/content/drive/MyDrive/ADD/challenge/0_Preliminary/0_Training/Pre_train_D_2_new.csv')
df.loc[: ,'id':'d7'] = df.loc[: ,'id':'d7'].apply(lambda x: x.astype(str).map(lambda x: int(x, 16)))
df.loc[: ,'id':'d7'] /= 256
df = df[['id', 'diff', 'Class', 'SubClass']]
test2 = np.array(df)


# # makeCSV('/content/drive/MyDrive/ADD/challenge/0_Preliminary/1_Submission/Pre_submit_D.csv', 0)

# df = pd.read_csv('/content/drive/MyDrive/ADD/challenge/0_Preliminary/1_Submission/Pre_submit_D_new.csv')
# df.loc[: ,'id':'d7'] = df.loc[: ,'id':'d7'].apply(lambda x: x.astype(str).map(lambda x: int(x, 16)))
# df.loc[: ,'id':'d7'] /= 256
# df = df[['id', 'diff', 'Class', 'SubClass']]
# test3 = np.array(df)

<ipython-input-2-48a499917398>:33: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[: ,'id':'d7'] = df.loc[: ,'id':'d7'].apply(lambda x: x.astype(str).map(lambda x: int(x, 16)))
<ipython-input-2-48a499917398>:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[: ,'id':'d7'] = df.loc[: ,'id':'d7'].apply(lambda x: x.astype(str).map(lambda x: int(x, 16)))
<ipython-input-2-48a499917398>:51: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split


N = 200
d = 50

def makeDataset(data):
    X, Y = list(), list()
    XX, YY = list(), list()
    for i in range(0, len(data), d):
        if i + N < len(data):
            temp = data[i:i+N]
            flag = 1
            for t in temp:
                if t[-2] == 'Attack':
                    a = np.array(temp[:, :-2]).astype(np.float32)
                    XX.append(a)
                    YY.append(t[-1])
                    flag = 0
                    break
            if flag:
                a = np.array(temp[:, :-2]).astype(np.float32)
                X.append(a)
                Y.append(t[-1])
    return np.array(X), np.array(Y), np.array(XX), np.array(YY)



nor1X, nor1Y, _, _ = makeDataset(free)
nor2X, nor2Y, att2X, att2Y = makeDataset(test1)
nor3X, nor3Y, att3X, att3Y = makeDataset(test2)

tr1, te1 = train_test_split(nor1X, test_size = 0.2)
tr2, te2 = train_test_split(nor2X, test_size = 0.2)
tr3, te3 = train_test_split(nor3X, test_size = 0.2)
tr = np.concatenate((tr1, tr2, tr3))
te = np.concatenate((te1, te2, te3))

In [ ]:
print(nor1X.shape, tr.shape)

(3583, 200, 2) (18267, 200, 2)


In [ ]:
keras

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv1D, Bidirectional, LSTM, ReLU, Dense

model = keras.Sequential([
    keras.Input(shape=(200, 2)),
    Conv1D(128, 3, 1, padding='same', activation='relu'),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(128, return_sequences=True)),
    ReLU(),
    Dense(2)
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 200, 128)          896       
                                                                 
 bidirectional_4 (Bidirectio  (None, 200, 256)         263168    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 200, 256)         394240    
 nal)                                                            
                                                                 
 re_lu_2 (ReLU)              (None, 200, 256)          0         
                                                                 
 dense_2 (Dense)             (None, 200, 2)            514       
                                                                 
Total params: 658,818
Trainable params: 658,818
Non-tr

In [ ]:
normal_train = tf.data.Dataset.from_tensor_slices((tr, tr)).batch(8)
normal_test = tf.data.Dataset.from_tensor_slices((te, te)).batch(1)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mae')

history = model.fit(tr, tr, batch_size=32, epochs=100, verbose=True)

In [ ]:
pred = model.predict(te) #모델 output 예측값

In [ ]:
model.evaluate(te, te, batch_size=1)

In [ ]:
model.evaluate(att2X, att2X, batch_size=1)